<a href="https://colab.research.google.com/github/midnightBlueNebula/book-recommendation/blob/main/fcc_book_recommendation_knn_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-06-20 08:16:12--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   109MB/s    in 0.2s    

2022-06-20 08:16:13 (109 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [19]:
# add your code here - consider creating a new cell for each section of code

user_counts = df_ratings["user"].value_counts()
isbn_counts = df_ratings["isbn"].value_counts()

df_filter = df_ratings

df_filter = df_filter[~df_filter['user'].isin(user_counts[user_counts < 200].index)]
df_filter = df_filter[~df_filter['isbn'].isin(isbn_counts[isbn_counts < 100].index)]

df = pd.merge(left=df_books, right=df_filter, on="isbn")
df.drop_duplicates(subset=["title", "user"], inplace=True, keep="first")

pivot_table = pd.pivot_table(df, index="title", columns="user", values="rating", fill_value=0)
title_list = list(pivot_table.index.values)

print(pivot_table)
print(title_list)

user                                                254     2276    2766    \
title                                                                        
1984                                                     9       0       0   
1st to Die: A Novel                                      0       0       0   
2nd Chance                                               0      10       0   
4 Blondes                                                0       0       0   
A Beautiful Mind: The Life of Mathematical Geni...       0       0       0   
...                                                    ...     ...     ...   
Without Remorse                                          0       0       0   
Year of Wonders                                          0       0       0   
You Belong To Me                                         0       0       0   
Zen and the Art of Motorcycle Maintenance: An I...       0       0       0   
\O\" Is for Outlaw"                                      0      

In [20]:
neighbors = NearestNeighbors(n_neighbors=5, metric="cosine", algorithm="brute")
neighbors.fit(pivot_table.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [21]:
# function to return recommended books - this will be tested

def get_index(title):
  return pivot_table.values[title_list.index(title)]


def get_recommends(book = ""):
  rating =  df[df["title"] == book].values[0][4]

  index = get_index("Where the Heart Is (Oprah's Book Club (Paperback))")
  distances, indices = neighbors.kneighbors([index])

  result = []
  recommended_books = [book, result]

  for distance, index in zip(distances[0], indices[0]):
    title = pivot.index.values[index]
    print(title)
    row = [title, distance]
    result.insert(0, row)

  return recommended_books

get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

Where the Heart Is (Oprah's Book Club (Paperback))
The Lovely Bones: A Novel
I Know This Much Is True
The Surgeon
The Weight of Water


["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Weight of Water', 0.7708583572697412],
  ['The Surgeon', 0.7699410973804288],
  ['I Know This Much Is True', 0.7677075092617776],
  ['The Lovely Bones: A Novel', 0.7234864549790632],
  ["Where the Heart Is (Oprah's Book Club (Paperback))",
   7.771561172376096e-16]]]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [22]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

Where the Heart Is (Oprah's Book Club (Paperback))
The Lovely Bones: A Novel
I Know This Much Is True
The Surgeon
The Weight of Water
["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7708583572697412], ['The Surgeon', 0.7699410973804288], ['I Know This Much Is True', 0.7677075092617776], ['The Lovely Bones: A Novel', 0.7234864549790632], ["Where the Heart Is (Oprah's Book Club (Paperback))", 7.771561172376096e-16]]]
Where the Heart Is (Oprah's Book Club (Paperback))
The Lovely Bones: A Novel
I Know This Much Is True
The Surgeon
The Weight of Water
You passed the challenge! 🎉🎉🎉🎉🎉
